# Google search try

In [ ]:
from googlesearch import search
import re
import pandas as pd

In [ ]:
companies = pd.read_csv('../data/processed/companies.csv')

In [ ]:
companies = companies['name'].tolist()

In [ ]:
# to search
found = set()

for idx,company in enumerate(companies):
    for j in search(company, tld="com"):
        print(idx)
        if re.search('wikipedia',j):
            found.add(company)
            break

In [ ]:
search

In [ ]:
import bs4 as bs
import urllib

comp = 'The Bank of East Asia Ltd'
url = 'https://en.wikipedia.org/wiki/'
print(urllib.parse.urlencode(comp))
# source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'html')

In [ ]:
for tag in soup.find_all('td'):
    if tag.get('class') == ['infobox-data', 'adr']:
        print(tag.text)

# Nominatim

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd 
import os
import sys
import ssl
import numpy as np


sys.path.append('../')

In [ ]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [ ]:
geolocator = Nominatim(user_agent="my-application3", ssl_context=ssl_context, timeout=None)

In [ ]:
# Random test
location = geolocator.geocode("175 5th Avenue NYC, United states")
location.latitude 

In [ ]:
address = pd.read_parquet('../data/raw/addresses.parquet.gzip')
country_codes  = pd.read_excel('../data/raw/Countries.xlsx')

In [ ]:
address['completed'] = np.nan 

In [ ]:
address['city'] = address['city_state_postal'].str.split(',').str[0]

In [ ]:
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Aachen'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Artarmon'), 'country'] = 'AUS'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Bangalore'), 'country'] = 'IND'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Blindern'), 'country'] = 'NOR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Boulder'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Cambridge'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Lausanne'), 'country'] = 'CHE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Munich'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'New York'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Norwich'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Paris'), 'country'] = 'FRA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Southborough'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'country'] = 'SWE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Towson'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Waltham'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'country'] = 'CHE'

# Adding the only other company with location_street1
address.loc[(address['name'] == 'Fennoa Oy'), 'city'] = 'Alavus'
address.loc[(address['name'] == 'Fennoa Oy'), 'country'] = 'FIN'
address.loc[(address['name'] == 'Fennoa Oy'), 'city_state_postal'] = 'Alavus, 63300'
address.loc[(address['name'] == 'Fennoa Oy'), 'location_street1'] = 'Mikko Kalliovaara Rinteentie 136'

In [ ]:
address.loc[(address.country == 'NAM'), 'country'] = None

In [ ]:
address = address.merge(country_codes, how='left', left_on='country', right_on='ISO-alpha3 Code')

In [ ]:
# Have to take care of this when finding the location
# address.loc[(~address['location_street1'].isnull())&(address.city_state_postal.isnull())] 

In [ ]:
address.drop(columns=['No','M49 Code','Region 1', 'Region 2', 'country'], inplace=True)
address.rename(columns={'Country or Area':'country'}, inplace=True)

In [ ]:
address.head(2)

### Getting the latitude and longitude from Nominatim

In [ ]:
def api_call(search_str):
    """
    This function takes a search string and returns a location object obtained from the Nomnatim API.

    Parameters:
    search_str (str): The search string to be used in the API call.

    Returns:
    location (object): The location object returned from the API call. 
    """
    try:
        location = geolocator.geocode(search_str)
        return location
    except:
        return None



def new_search_str(old_search_str, city_state_postal, location_street1, country):
    """
    This function modifies the search string to be used in the API call. If the search string is None, it creates one from the 
    city_state_postal, location_street1, and country columns. If the search string is not None, it modifies the search string.

    Parameters:
    old_search_str (str): The search string to be modified.
    city_state_postal (str): The city, state, and postal code of the company.
    location_street1 (str): The street address of the company.
    country (str): The country of the company.

    Returns:
    search_str (str): The modified search string.

    """

    if old_search_str is None:

        # we create a new search string

        if country is None:
            search_str = None
        
        else:
            if (city_state_postal is None) & (location_street1 is None):
                search_str = country
            
            if (city_state_postal is None) & (location_street1 is not None):
                search_str = location_street1 + ', ' + country
            
            if (city_state_postal is not None) & (location_street1 is None):
                search_str = city_state_postal + ', ' + country

            if (city_state_postal is not None) & (location_street1 is not None):
                search_str = location_street1 + ', ' +  city_state_postal + ', ' + country

    else:

        # If the old search string doesnt give proper results, we modify it
        # we remove the first word from the search string and try again

        search_str = old_search_str[old_search_str.find(" ")+1:]

    return search_str


def get_location(row):
    """
    This function takes a row from the address dataframe and returns a location object obtained from the Nomnatim API.

    Parameters:
    row (series): A row from the address dataframe.

    Returns:
    location (object): The location object returned from the API call. 
    """
    old_search_str = None
    x = None 

    while x is None:

        search_str = new_search_str(old_search_str, rows['city_state_postal'], rows['location_street1'], rows['country'])
        x = api_call(search_str)
        
        if x is None:
            old_search_str = search_str
            completed = 0
            continue

        if x is not None:
 
            completed = 1
            break

        if search_str == old_search_str:
            completed = 1
            break

    return completed, x

In [ ]:
address = pd.read_parquet('../data/processed/addresses1.parquet.gzip')

In [ ]:
address_new = address.loc[(address['completed'] != 1)]

for id, rows in address_new.iterrows():

    print(id)

    if rows['completed'] != 1:

        old_search_str = None
        x = None 

        while x is None:

            search_str = new_search_str(old_search_str, rows['city_state_postal'], rows['location_street1'], rows['country'])
            x = api_call(search_str)
            
            if x is None:
                old_search_str = search_str
                address.loc[id,'completed'] = 0
                continue

            if x is not None:
                lat = x.latitude
                log = x.longitude
                address.loc[id,'lat'] = lat
                address.loc[id,'log'] = log
                address.loc[id,'completed'] = 1
                break

            if search_str == old_search_str:
                address.loc[id,'completed'] = 1
                break
            # i = 1
            # while i < len(rows['city_state_postal'].split(',')) :
            #     search_str = ','.join(rows['city_state_postal'].split(',')[0:i])+", " + rows['country']
            #     print(search_str)
            #     x = geolocator.geocode(search_str)
            #     if x is None:
            #         i += 1
            #         continue
            #     if x is not None:
            #         lat = x.latitude
            #         log = x.longitude
            #         address.loc[id,'lat'] = lat
            #         address.loc[id,'log'] = log
            #         address.loc[id,'completed'] = 1
            #         break
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses1.parquet.gzip', compression='gzip')
        print('saved')

In [ ]:
address = pd.read_parquet('../data/processed/addresses1.parquet.gzip')
address.loc[(address.location_street1.isnull())] 

In [ ]:
address = pd.read_parquet('../data/processed/addresses.parquet.gzip')
address

In [ ]:
address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')

In [ ]:
address.country.isna().sum()

In [ ]:
country_codes 

In [ ]:
uni = address.groupby('city').agg(
    unique_country = pd.NamedAgg('country',lambda x: x.unique().tolist())
).reset_index()

In [ ]:
mult = uni.loc[uni['unique_country'].apply(lambda x: len(x)) > 1]

In [ ]:
mult_city = mult.explode('unique_country')

In [ ]:
address = mult_city.merge(country_codes, left_on='unique_country', right_on='Alpha-3 code', how='left')

In [ ]:
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Aachen'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Artarmon'), 'Country'] = 'Australia'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Bangalore'), 'Country'] = 'India'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Boulder'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Cambridge'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Lausanne'), 'Country'] = 'Switzerland'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Munich'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'New York'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Norwich'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Paris'), 'Country'] = 'France'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Southborough'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'Country'] = 'Sweden'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Towson'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Waltham'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'Country'] = 'Switzerland'

In [ ]:
address

In [ ]:
for id, row in address.iterrows():
    print(id)
    mask = (address['city'] == row['city'])&(address['country'] == row['unique_country'])

    x = geolocator.geocode(row['city']+ ", "+ row['Country'])
    
    if x is not None:

        lat = x.latitude
        log = x.longitude

        address.loc[mask,'lat'] = lat
        address.loc[mask,'log'] = log

    if x is None:

        address.loc[mask, 'lat'] = None
        address.loc[mask, 'log'] = None
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')
    

In [ ]:
address.to_csv('../data/processed/addresses.csv')